# Building an AI Flight Search Agent with Hyperbrowser

This cookbook demonstrates how to create an intelligent flight search agent that can find and extract structured flight information from travel websites. Using Hyperbrowser's Browser Use capabilities with vision enabled, this agent can:

- Navigate to Google Flights
- Enter specific departure and return dates
- Set origin and destination cities
- Extract detailed flight information in a structured format
- Return JSON-formatted results for easy processing

Unlike traditional web scraping techniques that can break when websites update their UIs, this approach leverages AI to interact with the site just like a human would. The agent can visually interpret the flight search results page and extract precisely structured data using a predefined Pydantic model.


## Prerequisites

Before starting, you'll need:

1. A Hyperbrowser API key (sign up at hyperbrowser.ai if you don't have one)
2. Python 3.9+ installed
3. The dotenv package for loading environment variables
4. The pydantic package for structured data modeling

Store your API key in a .env file in the same directory as this notebook:

```
HYPERBROWSER_API_KEY=your_hyperbrowser_key_here
```


## Step 1: Import Libraries and Set Up Environment

First, we import the necessary packages and initialize our environment. The key components include:

- `AsyncHyperbrowser`: For asynchronous communication with the Hyperbrowser API
- `StartBrowserUseTaskParams`: To configure the autonomous browser interaction
- `IPython.display`: For displaying results in the notebook

This setup allows us to create a powerful flight search experience that extracts structured data from visual flight search results.

In [ ]:
import os

from dotenv import load_dotenv
from hyperbrowser import AsyncHyperbrowser
from hyperbrowser.models import StartBrowserUseTaskParams

from IPython.display import Markdown, display
from pydantic import BaseModel

load_dotenv()

## Step 2: Initialize the Hyperbrowser Client

Next, we create an instance of the AsyncHyperbrowser client using our API key. This client will handle all communication with the Hyperbrowser API and allow us to create browser sessions and invoke the Browser Use agent with vision capabilities.


In [15]:
hb = AsyncHyperbrowser(api_key=os.getenv("HYPERBROWSER_API_KEY"))

## Step 3: Define the Data Model and Flight Search Function

Now we'll create a structured data model for flight details using Pydantic and define our flight search function. The FlightDetails model specifies the exact structure we want for each flight result, including:

- Departure and arrival times
- Flight duration
- Price information
- Airline name
- Number of stops

The search function uses the browser_use agent with vision enabled (use_vision=True) to:

1. Navigate to Google Flights
2. Search for flights between SF and NY for specific dates
3. Extract the flight details in the exact structured format defined by our model
4. Return the results as formatted JSON

Often times, the agent has only limited information about the page from the textual representation. Adding the `useVision` param does take up more tokens, but enables the agent to recognise more features in the page, such as there, there being more elements in page, and so that the agent should scroll down to collect all the information.

In [16]:
class FlightDetails(BaseModel):
    departure_time: str
    arrival_time: str
    duration: str
    price: str
    airline: str
    stops: str


async def summarize_flight_results():
    resp = await hb.agents.browser_use.start_and_wait(
        StartBrowserUseTaskParams(
            task=f"Go to google flights and search for flights from SF to NY on the 16th of March and returning on the 20th of March 2025. Return the flight details in a json format like this {FlightDetails.model_json_schema()}",
            use_vision=True,
        )
    )
    if resp.data is not None:
        return resp.data.final_result
    return None

## Step 4: Execute the Flight Search and Display Results

Finally, we run our flight search function and display the results as Markdown. The agent will:

1. Open a new browser session automatically
2. Navigate to Google Flights
3. Enter San Francisco as the origin city
4. Enter New York as the destination city
5. Set departure date to March 16, 2025
6. Set return date to March 20, 2025
7. Wait for search results to load
8. Visually interpret the flight search results page
9. Extract all flight options with their details
10. Format the information according to our FlightDetails model

The result is a comprehensive list of available flights with all relevant details in a structured JSON format.


In [17]:
response = await summarize_flight_results()
if response is not None:
    display(Markdown(response))
else:
    print("No response from the agent")

[{'departure_time': '11:52 PM', 'arrival_time': '3:01 PM+1', 'duration': '12 hr 9 min', 'price': '$261', 'airline': 'Frontier', 'stops': '1 stop in ATL'}, {'departure_time': '6:31 AM', 'arrival_time': '3:00 PM', 'duration': '5 hr 29 min', 'price': '$658', 'airline': 'AlaskaHawaiian', 'stops': 'Nonstop'}, {'departure_time': '7:27 AM', 'arrival_time': '4:01 PM', 'duration': '5 hr 34 min', 'price': '$658', 'airline': 'American', 'stops': 'Nonstop'}, {'departure_time': '11:02 AM', 'arrival_time': '6:30 AM+1', 'duration': '16 hr 28 min', 'price': '$261', 'airline': 'Frontier', 'stops': '1 stop in LAS'}, {'departure_time': '6:49 PM', 'arrival_time': '2:53 PM+1', 'duration': '17 hr 4 min', 'price': '$603', 'airline': 'Alaska, Spirit', 'stops': '1 stop in LAS'}, {'departure_time': '11:20 PM', 'arrival_time': '10:04 AM+1', 'duration': '7 hr 44 min', 'price': '$631', 'airline': 'JetBlue', 'stops': '1 stop in BOS'}, {'departure_time': '7:20 AM', 'arrival_time': '3:46 PM', 'duration': '5 hr 26 min', 'price': '$643', 'airline': 'AlaskaHawaiian, American', 'stops': 'Nonstop'}, {'departure_time': '9:01 AM', 'arrival_time': '5:30 PM', 'duration': '5 hr 29 min', 'price': '$658', 'airline': 'AlaskaHawaiian', 'stops': 'Nonstop'}, {'departure_time': '10:37 AM', 'arrival_time': '7:13 PM', 'duration': '5 hr 36 min', 'price': '$658', 'airline': 'American', 'stops': 'Nonstop'}, {'departure_time': '1:20 PM', 'arrival_time': '9:55 PM', 'duration': '5 hr 35 min', 'price': '$658', 'airline': 'American', 'stops': 'Nonstop'}, {'departure_time': '10:52 PM', 'arrival_time': '7:29 AM+1', 'duration': '5 hr 37 min', 'price': '$658', 'airline': 'American', 'stops': 'Nonstop'}, {'departure_time': '7:15 AM', 'arrival_time': '3:55 PM', 'duration': '5 hr 40 min', 'price': '$658', 'airline': 'United', 'stops': 'Nonstop'}, {'departure_time': '10:45 AM', 'arrival_time': '7:09 PM', 'duration': '5 hr 24 min', 'price': '$658', 'airline': 'United', 'stops': 'Nonstop'}, {'departure_time': '12:55 PM', 'arrival_time': '9:14 PM', 'duration': '5 hr 19 min', 'price': '$658', 'airline': 'United', 'stops': 'Nonstop'}, {'departure_time': '1:55 PM', 'arrival_time': '10:10 PM', 'duration': '5 hr 15 min', 'price': '$658', 'airline': 'United', 'stops': 'Nonstop'}, {'departure_time': '2:45 PM', 'arrival_time': '11:15 PM', 'duration': '5 hr 30 min', 'price': '$658', 'airline': 'United', 'stops': 'Nonstop'}, {'departure_time': '4:40 PM', 'arrival_time': '1:12 AM+1', 'duration': '5 hr 32 min', 'price': '$658', 'airline': 'United', 'stops': 'Nonstop'}, {'departure_time': '9:30 PM', 'arrival_time': '5:57 AM+1', 'duration': '5 hr 27 min', 'price': '$658', 'airline': 'United', 'stops': 'Nonstop'}, {'departure_time': '10:55 PM', 'arrival_time': '7:17 AM+1', 'duration': '5 hr 22 min', 'price': '$658', 'airline': 'United', 'stops': 'Nonstop'}, {'departure_time': '11:55 PM', 'arrival_time': '8:11 AM+1', 'duration': '5 hr 16 min', 'price': '$658', 'airline': 'United', 'stops': 'Nonstop'}, {'departure_time': '7:10 AM', 'arrival_time': '3:30 PM', 'duration': '5 hr 20 min', 'price': '$713', 'airline': 'Delta', 'stops': 'Nonstop'}, {'departure_time': '9:15 AM', 'arrival_time': '5:42 PM', 'duration': '5 hr 27 min', 'price': '$713', 'airline': 'Delta', 'stops': 'Nonstop'}, {'departure_time': '2:35 PM', 'arrival_time': '11:00 PM', 'duration': '5 hr 25 min', 'price': '$713', 'airline': 'Delta', 'stops': 'Nonstop'}, {'departure_time': '4:05 PM', 'arrival_time': '12:30 AM+1', 'duration': '5 hr 25 min', 'price': '$713', 'airline': 'Delta', 'stops': 'Nonstop'}, {'departure_time': '10:50 PM', 'arrival_time': '7:14 AM+1', 'duration': '5 hr 24 min', 'price': '$713', 'airline': 'Delta', 'stops': 'Nonstop'}, {'departure_time': '6:00 AM', 'arrival_time': '2:29 PM', 'duration': '5 hr 29 min', 'price': '$722', 'airline': 'JetBlue', 'stops': 'Nonstop'}, {'departure_time': '10:44 AM', 'arrival_time': '7:19 PM', 'duration': '5 hr 35 min', 'price': '$722', 'airline': 'JetBlue', 'stops': 'Nonstop'}, {'departure_time': '8:43 PM', 'arrival_time': '5:10 AM+1', 'duration': '5 hr 27 min', 'price': '$722', 'airline': 'JetBlue', 'stops': 'Nonstop'}, {'departure_time': '9:00 AM', 'arrival_time': '5:20 PM', 'duration': '5 hr 20 min', 'price': '$769', 'airline': 'United', 'stops': 'Nonstop'}, {'departure_time': '2:00 PM', 'arrival_time': '10:28 PM', 'duration': '5 hr 28 min', 'price': '$787', 'airline': 'JetBlue', 'stops': 'Nonstop'}, {'departure_time': '4:40 PM', 'arrival_time': '12:57 AM+1', 'duration': '5 hr 17 min', 'price': '$787', 'airline': 'JetBlue', 'stops': 'Nonstop'}, {'departure_time': '11:55 AM', 'arrival_time': '8:29 PM', 'duration': '5 hr 34 min', 'price': '$848', 'airline': 'Delta', 'stops': 'Nonstop'}]

## Data Analysis and Insights

Looking at the flight search results, we can observe several interesting patterns:

1. Price Distribution: The cheapest flights are around $261 (Frontier), but they involve stops and significantly longer travel times. Nonstop flights start at around $658.
2. Time vs. Price Tradeoff: Flights with stops can save about $400, but add 6-12 hours to the journey time.
3. Airline Patterns:
    - United offers the most frequent nonstop service
    - Budget carriers like Frontier offer the lowest prices but with connections
    - Premium carriers like Delta price their nonstop flights higher at about $713
4. Time of Day Options: There are flight options throughout the day with early morning, mid-day, and evening/red-eye options available.

The structured data format makes it easy to perform further analysis, such as filtering for the shortest duration flights, sorting by price, or comparing airlines.


## Extensions and Applications

This flight search agent can be extended in several powerful ways:

1. Expanded Data Model: Add fields for baggage allowance, seat type, or aircraft model
2. Multi-site Comparison: Search across multiple travel sites (Google Flights, Kayak, Expedia) and compare results
3. Filtering and Sorting: Add post-processing to filter results by preferred airlines, maximum price, or minimum layover time
4. Fare Tracking: Set up periodic searches to track price changes for specific routes
5. Booking Workflow: Extend the agent to proceed through the booking flow to a specific point, then hand off to a human user
6. Travel Package Search: Expand to search for hotel+flight packages with similar structured output

These extensions can transform this simple flight search into a comprehensive travel planning assistant.


## Conclusion

We've demonstrated how to build an intelligent flight search agent using Hyperbrowser's browser_use capabilities with vision enabled. This approach offers several advantages:

- Robustness: The AI can handle dynamic websites and UI changes that would break traditional scrapers
- Structured Data: The Pydantic model ensures consistent, well-formatted results
- Easy Adaptation: The same approach can be applied to other travel sites or search criteria
- Automated Visual Interpretation: Using use_vision=True allows the agent to interpret complex search results visually, and realise more of the options available to the agent.

This pattern demonstrates how AI agents can automate complex web interactions that previously required human effort, saving time and enabling programmatic access to travel information that might not be available through official APIs.


## Relevant Links
- [Hyperbrowser](https://hyperbrowser.ai)
- [Hyperbrowser Documentation](https://docs.hyperbrowser.ai)
- [Hyperbrowser Browser Use Documentation](https://docs.hyperbrowser.ai/agents/browser-use)